<a href="https://colab.research.google.com/github/aruzhan-bolatova/HolidayRate_data_feature/blob/main/Data_feature_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This **HolidayRate** data feature provides a handy tool for travelers looking to **find the cheapest flights during holidays**. It leverages two powerful APIs, Calendarific and Amadeus, to achieve this:

- [Calendarific API](https://calendarific.com/): This API is used to fetch a list of holidays for a specified country and month. The notebook prompts the user to input the country and month they are interested in and retrieves the corresponding holiday data.

- [Amadeus API](https://developers.amadeus.com/self-service): This API is the backbone of the flight search functionality. Once the holidays are identified, the notebook uses the Amadeus API to retrieve flight information for each holiday date. This includes details like departure and arrival airports, flight dates, and, most importantly, the total price of the flight.

**Key Features:**

- Holiday-Specific Flight Prices: The tool retrieves flight prices specifically for the holiday dates, allowing users to see how prices fluctuate during these peak travel times.

- Price Comparison: Users can compare flight prices for different holidays within the same month to identify the most affordable travel dates.

- Normal Date Comparison: The tool also allows users to input a non-holiday date to compare flight prices and see the potential cost savings of traveling outside of holiday periods.

This data feature empowers travelers to make informed decisions about their travel plans, potentially saving them money by identifying cheaper flights during holidays.

**Future Improvements:**

- Adding a feature to automatically consider the next year if the selected month has already passed in the current year without a need to specifically input a year

- Customize Flight Offers: Number of passengers, Cabin class, Number of stops, Airlines

- Improve IATA Code Retrieval: The current method for obtaining IATA codes is scraping a website, which has errors in its data (i.e. Astana - 'TSE' is incorrect, should be 'NQZ').
Investigate alternative data sources for IATA codes.

- Data Validation: Implement checks to ensure that the user inputs are valid, such as checking if the country code exists,validating the date format, confirming that the IATA codes are valid

In [30]:
#setting up API keys and testing if they can be successfully retrieved
from google.colab import userdata
import requests
import json

CALENDARIFIC_API_KEY = userdata.get('CALENDARIFIC_API_KEY')
AMADEUS_API_KEY = userdata.get('AMADEUS_API_KEY')

if CALENDARIFIC_API_KEY:
  print("CALENDARIFIC_API_KEY is set")
else:
  print("CALENDARIFIC_API_KEY is not set")

if AMADEUS_API_KEY:
  print("AMADEUS_API_KEY is set")
else:
  print("AMADEUS_API_KEY is not set")

CALENDARIFIC_API_KEY is set
AMADEUS_API_KEY is set


In [48]:
#Function to get access token for Amadeus API
def get_access_token():

  # Set the credentials
  AMADEUS_API_SECRET = userdata.get('AMADEUS_API_SECRET')
  AMADEUS_API_KEY = userdata.get('AMADEUS_API_KEY')

  # Define the URL and the request parameters
  url = "https://test.api.amadeus.com/v1/security/oauth2/token"
  headers = {
      "Content-Type": "application/x-www-form-urlencoded"
  }
  data = {
      "grant_type": "client_credentials",
      "client_id": AMADEUS_API_KEY,
      "client_secret": AMADEUS_API_SECRET
  }

  # Make the request to get the access token
  response = requests.post(url, headers=headers, data=data)

  # Check if the request was successful
  if response.status_code == 200:
      # Extract the access token from the response JSON
      access_token = response.json().get('access_token')
      # print(f"Access token: {access_token}")
      return access_token
  else:
      print(f"Failed to get access token: {response.status_code}, {response.text}")
      return None


In [49]:
#Setting up access to Google Sheets from Colab notebook
!pip install --upgrade gspread
from google.colab import auth
auth.authenticate_user()

In [50]:
#Extracts information about country names and codes from Google Sheets
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet
sheet = gc.open('countries code').sheet1

# Get all values from the sheet
data = sheet.get_all_values()

# Convert to a dictionary
country_codes = {row[0]: row[1] for row in data}

print(country_codes)

{'Afghanistan': 'af', 'Albania': 'al', 'Algeria': 'dz', 'American Samoa': 'as', 'Andorra': 'ad', 'Angola': 'ao', 'Anguilla': 'ai', 'Antigua and Barbuda': 'ag', 'Argentina': 'ar', 'Armenia': 'am', 'Aruba': 'aw', 'Australia': 'au', 'Austria': 'at', 'Azerbaijan': 'az', 'Bahrain': 'bh', 'Bangladesh': 'bd', 'Barbados': 'bb', 'Belarus': 'by', 'Belgium': 'be', 'Belize': 'bz', 'Benin': 'bj', 'Bermuda': 'bm', 'Bhutan': 'bt', 'Bolivia': 'bo', 'Bosnia and Herzegovina': 'ba', 'Botswana': 'bw', 'Brazil': 'br', 'British Virgin Islands': 'vg', 'Brunei': 'bn', 'Bulgaria': 'bg', 'Burkina Faso': 'bf', 'Burundi': 'bi', 'Cabo Verde': 'cv', 'Cambodia': 'kh', 'Cameroon': 'cm', 'Canada': 'ca', 'Cayman Islands': 'ky', 'Central African Republic': 'cf', 'Chad': 'td', 'Chile': 'cl', 'China': 'cn', 'Colombia': 'co', 'Comoros': 'km', 'Congo': 'cg', 'Congo Democratic Republic': 'cd', 'Cook Islands': 'ck', 'Costa Rica': 'cr', "Cote d'Ivoire": 'ci', 'Croatia': 'hr', 'Cuba': 'cu', 'Curaçao': 'cw', 'Cyprus': 'cy', 'Cze

In [77]:
#Function to retrieve holidays information from Calendarific API based on input country and month from user
def get_holiday_data(country_code, input_year, input_month):
  url = f"https://calendarific.com/api/v2/holidays?api_key={CALENDARIFIC_API_KEY}&country={country_code}&year={input_year}&month={input_month}"
  response = requests.get(url)
  data = response.json()
  #print(data)

  # Extracting the holidays from the response
  holidays = data['response']['holidays']

  # Printing the holiday name and date in a list format
  counter = 0
  dates = []
  for holiday in holidays:
    counter += 1
    name = holiday['name']
    date = f"{holiday['date']['datetime']['year']:04d}-{holiday['date']['datetime']['month']:02d}-{holiday['date']['datetime']['day']:02d}"
    dates.append(date)
    print(f"{counter}) {name}, Date: {date}")
  return dates #further used to pass to Amandeus API and retrieve flight tickets information


In [83]:
import requests
from bs4 import BeautifulSoup

#Function to find IATA airport codes based on city or country name  (Resource: https://stackoverflow.com/questions/62494384/finding-name-and-codes-of-all-airports)
def get_iata_codes(country_name):
  # Fetch the page
  page = requests.get('http://www.airportcodes.org/')
  soup = BeautifulSoup(page.text, 'lxml')

  # Find the div or table containing the IATA codes and country names
  # Assuming there's a div with class "i1" where this data is stored
  table = soup.find('div', attrs={"class": "i1"})
  # print(table)

  # Initialize a list to store the country names and IATA codes
  iata_codes = []

  # Assuming that the content inside the div is in the format "City, Country (IATA Code)"
  for line in table.text.splitlines():
      line = line.strip()  # Remove leading/trailing whitespace
      if line:  # If the line is not empty
          # Split the text into the city-country part and the IATA code part
          parts = line.split('(')
          if len(parts) == 2:
              location = parts[0].strip()  # City, Country
              iata_code = parts[1].strip(')')  # IATA Code without parentheses
              iata_codes.append((location, iata_code))
  # print(iata_codes)

  # Filter by country name or city name (for example, "Russia")
  for location, iata_code in iata_codes:
      if country_name in location:
          print(f"{location}, IATA Code: {iata_code}")
          return iata_code


In [110]:
#Function to retrieve information from Amadeus API based on user's input of origin city, destination and holiday dates from Calenarific API
def get_flight_data(origin_code, destination_code, date):
  url = f"https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode={origin_code}&destinationLocationCode={destination_code}&departureDate={date}&adults=1&nonStop=false&max=3"
  access_token = get_access_token()
  headers = {
      "Authorization": f"Bearer {access_token}"
  }
  response = requests.get(url, headers=headers)
  data = response.json()

  # Extracting required details
  if data['data'] != []:
    departure_airport = data['data'][0]['itineraries'][0]['segments'][0]['departure']['iataCode']
    arrival_airport = data['data'][0]['itineraries'][0]['segments'][-1]['arrival']['iataCode']
    departure_date = data['data'][0]['itineraries'][0]['segments'][0]['departure']['at'][:10]  # Just taking date part
    total_price = data['data'][0]['price']['total']
    currency = data['dictionaries']['currencies']['EUR']

    # Creating the sentence
    sentence = f"The flight departs from {departure_airport} and arrives at {arrival_airport} on {departure_date}. The total price is {total_price} {currency}."
    print(sentence)
  else:  #Error handling in case flights are not found
    print("No flights found for the given criteria. Potential reason: outdated or incorrect IATA code information")

In [112]:
1#Putting everything together to create a feature

import requests
print("Please search for future dates (this year or upcoming year)!\n")
input_year = input("Enter a year (YYYY): ")
input_month = input("Enter a month (1-12): ")
country = input("Enter a country name: ")
country_code = country_codes.get(country)
# print(country_code)
dates = get_holiday_data(country_code, input_year, input_month)

origin_city = input("Enter the origin city: ")
destination_city = input("Enter the destination city: ")

origin_code = get_iata_codes(origin_city)
destination_code = get_iata_codes(destination_city)

for date in dates:
  print(date)
  get_flight_data(origin_code, destination_code, date)

date_free = input("Enter a date to compare (YYYY-MM-DD): ")
get_flight_data(origin_code, destination_code, date_free)



Please search for future dates (this year or upcoming year)!

Enter a year (YYYY): 2024
Enter a month (1-12): 12
Enter a country name: Russia
1) December Solstice, Date: 2024-12-21
2) Working Day, Date: 2024-12-28
3) New Year Holiday, Date: 2024-12-30
4) New Year Holiday, Date: 2024-12-31
Enter the origin city: Moscow
Enter the destination city: Rome
Moscow, Russia - all locations, IATA Code: MOW
Rome, Italy - All airports, IATA Code: ROM
2024-12-21
The flight departs from VKO and arrives at FCO on 2024-12-21. The total price is 374.46 EURO.
2024-12-28
The flight departs from DME and arrives at FCO on 2024-12-28. The total price is 378.10 EURO.
2024-12-30
The flight departs from DME and arrives at FCO on 2024-12-30. The total price is 350.50 EURO.
2024-12-31
The flight departs from DME and arrives at FCO on 2024-12-31. The total price is 350.50 EURO.
Enter a date to compare (YYYY-MM-DD): 2024-12-15
The flight departs from DME and arrives at FCO on 2024-12-15. The total price is 378.10 